# `Coding problem (100 points)`

## `(a)` Below you will find `7 questions`. Some will require you to do simple derivations and provide discussion. Some will require you to complete an existing code. 

## You can use all of the codes that we have discussed in the course to help your work. This is an open-book exam.

## `(b)` Your submission (on Canvas) at the end of this portion of the exam should consist of the following: (1) a scanned piece of paper with all of your derivations and discussions (partial credits will depend on your handwriting being as legible as possible) and (2) this Python code with your coding work. `Please note:` I am expecting `2 documents` with your submission.

## `(c)` Make sure you give yourself plenty of time to scan your work and submit online. The length of the exam is 2 hours starting with the quiz. 

###############################################################################################

# Maximum likelihood estimation of an AR(1)-GARCH(1,1) model 

### This code estimates the parameters ($\mu^{\ast}$, $\delta^{\ast}$, $\phi^{\ast}$, $\alpha$ and $\beta$) of an AR(1)-GARCH(1,1) model by maximizing the standardized (by $T$) log likelihood:

\begin{equation*}
Q_T(\theta) = \frac{1}{T}\log(L(\{x\},\mu^{\ast}, \delta^{\ast}, \phi^{\ast}, \alpha, \beta))= \frac{1}{2}\log (2\pi)-\frac{\sum _{t=3}^{T}}{2T}\log 
h_{t}-\frac{\sum _{t=3}^{T}}{2T}\left( \frac{(r _{t}-\alpha - \beta r_{t-1})^{2}}{h_{t}}\right),
\end{equation*}
with
$$h_t = \mu ^{\ast }+\delta ^{\ast }h_{t-1}+\phi^{\ast } (r_{t-1} -\alpha - \beta r_{t-2})^{2}.$$

##################################################################################################

# `Question 1 (Theory) (10 points)`
## Compute the unconditional expected value of returns ($\mathbb{E}(r_t)$) in this model. (Write the answer on your paper.)

# `Question 2 (Theory) (10 points)`
## Compute the conditional expected value of returns ($\mathbb{E}_{t-1}(r_t)$) in this model. Does it time-vary? (Write the answer on your paper.)

# `Code`

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
from scipy.stats import t, chi2, norm
from platform import python_version

In [ ]:
# The recommended python version is 3.8 or 3.9
print(python_version())
# Adjust the plot size on your computer
plt.rcParams['figure.figsize'] = [7, 7]
# Check current directory
os.getcwd()

## Let us begin by uploading the data

In [ ]:
data = pd.read_excel('SP500daily_level.xlsx')
data 

## Let us now extract prices and compute continuously-compounded returns

In [ ]:
# We go from prices to log (or continuously-compounded) returns
prices = np.array(data.iloc[:, 1])
ret = np.log(prices[1:] / prices[:-1])
T = len(ret)

## Always plot the data (if you can): why do we need a GARCH model? Let's see ...

In [ ]:
plt.plot(ret)
plt.title('SP 500 continuously-compounded returns', color = 'red')
plt.ylabel('returns', color = 'red')
plt.xlabel('time', color = 'red')

# `Question 3 (Coding) (15 points)`
## Estimate $\alpha$ and $\beta$ by linear least squares ignoring GARCH effects.

In [ ]:


    ##################################################
    # COMPLETE THE CODE BELOW
    ##################################################
    
    X = np.column_stack((np.ones(T-1), ret[1:]))
    estimates_ls = 
    fitted_values = 
    residuals =
    
    ##################################################
    # END OF CODE TO BE MODIFIED
    ##################################################


# `Question 4 (Coding) (10 points)`
## Report the least-squares estimates of $\alpha$ and $\beta$ and their t-statistics in a table. 

In [ ]:
    ##################################################
    # COMPLETE THE CODE BELOW
    ##################################################
    
    var_residuals = 
    VarCov_ls =
    std_errors_ls = 
    t_stats_ls = 
    
    ##################################################
    # END OF CODE TO BE MODIFIED
    ##################################################


table = pd.DataFrame({'Estimates LS': estimates_ls, 't statistics': t_stats_ls}, index = ['alpha', 'beta'])
table

# Next, we do GARCH

## (1) We define the standardized log-likelihood function

In [ ]:
def garchloglik(theta,ret):
    
    [mu, delta, phi, alpha, beta] = theta
    
    h1 = np.var(ret, ddof=1)          # Recall h needs to be initiated (we loop over h)
                                      # Initial value of h: we use the unconditional empirical variance from the data

    h    =   h1
    sum1 = - 1 / 2 * np.log(h1) /T
    sum2 = - (ret[1]- alpha - beta*ret[0])**2 / (2 * h1 * T)

    for t in range(2, T): 
        h = mu + delta * h + phi * (ret[t-1]- alpha - beta*ret[t-2])**2
        sum1 = sum1 - 1 / 2 * np.log(h)/T
        sum2 = sum2 - (ret[t] - alpha - beta*ret[t-1])**2 / (2 * h * T)

    # Note 1: I am excluding the first term in the log likelihood since it does not affect the maximum
    # Note 2: Below I will change the sign of the likelihood. We are maximizing, not minimizing.
    
    y = - (sum1 + sum2)
    return y

## (2) Estimation

The ML estimates are:

\begin{eqnarray*}
\widehat{\theta }_{MLE}&=&\underset{\theta}{\arg \max }\left[Q_{T}(\theta )\right] \\
&=& \underset{\theta = (\mu^{\ast}, \delta^{\ast}, \phi^{\ast}, \alpha, \beta)}{\arg \max }{\frac{1}{T}\log(L(\{x\},\mu^{\ast}, \delta^{\ast}, \phi^{\ast}, \alpha, \beta))}\\
&=&\underset{\theta = (\mu^{\ast}, \delta^{\ast}, \phi^{\ast}, \alpha, \beta) }{\arg \max }\left(\frac{1}{2}\log (2\pi)-\frac{\sum _{t=3}^{T}}{2T}\log 
h_{t}-\frac{\sum _{t=3}^{T}}{2T}\left( \frac{(r _{t}-\alpha - \beta r_{t-1})^{2}}{h_{t}}\right)\right)
\end{eqnarray*}

In [ ]:
theta_guess = [0, 0.5, 0.5, 0.1, 0.2]  

estimates = scipy.optimize.fmin(func=garchloglik, 
                                x0=theta_guess, 
                                args=(ret,), 
                                xtol=1e-15, 
                                ftol=1e-15, 
                                disp=0)  

table = pd.DataFrame({'Estimates':estimates}, index = ['mu', 'delta','phi', 'alpha', 'beta'])
table

# `Question 5 (Theory and Coding) (25 points)`
## Write the matrix $\Omega_0$ and code up its estimator below. You goal is to compute the standard errors of the parameter estimates. (The expression for the matrix $\Omega_0$ should be provided on your paper. The implementation should be in the code.)

In [ ]:
h = np.var(ret, ddof=1) * np.ones(T)
grad1_h = np.zeros(T)
grad2_h = np.zeros(T)
grad3_h = np.zeros(T)
grad4_h = np.zeros(T)
grad5_h = np.zeros(T)
Omegahat = np.zeros([5, 5])

    ##################################################
    # COMPLETE THE CODE BELOW
    ##################################################

for t in range(1, T):
    h[t] = 
    grad1_h[t] = 
    grad2_h[t] = 
    grad3_h[t] = 
    grad4_h[t] = 
    grad5_h[t] = 

    grad_h = np.array([grad1_h[t], grad2_h[t], grad3_h[t], grad4_h[t], grad5_h[t]]).T

    grad = 
    grad[3] = 
    grad[4] = 
    Omegahat = 

    ##################################################
    # END OF CODE TO BE MODIFIED
    ################################################## 
    

In [ ]:
VarCov = (1 / T) * np.linalg.inv(Omegahat)
t_stats = estimates / np.sqrt(np.diag(VarCov))

table = pd.DataFrame({'Estimates': estimates, 't statistics': t_stats}, index = ['mu', 'delta','phi', 'alpha', 'beta'])
table

# `Question 6 (Theory and Coding) (15 points)`
## Test if $\beta = 0.05$ using the appropriate test statistic and the appropriate p-value. (Write the test on your paper and implement the test in the code.)

In [ ]:
    ##################################################
    # COMPLETE THE CODE BELOW
    ##################################################

    
t_test = 
p_value = 


    ##################################################
    # END OF CODE TO BE MODIFIED
    ##################################################  

# `Question 7 (Theory and Coding) (15 points)`
## Test if $\delta^{\ast} = 0.9$ and $\phi^{\ast} = 0.07$ using the appropriate test statistic and the appropriate p-value. (Write the test on your paper and implement the test in the code.)  

In [ ]:
    ##################################################
    # COMPLETE THE CODE BELOW
    ##################################################

R = 
r = 

test_statistic = 
p_value = 

    ##################################################
    # END OF CODE TO BE MODIFIED
    ################################################## 